In [1]:
import os
import sys
sys.path.append('..')
from data_center.tej_handler import TEJHandler
from monitor.tw_monitor import TWMarketMonitor

from pymongo import MongoClient
import tejapi

import pandas as pd
import numpy as np
from datetime import date
from dateutil.relativedelta import relativedelta
# TEJHandler()

In [7]:
TEJHandler().get_tej_data_table()

,0
融資券資料,TWN/EWGIN
單季財務資料表,TWN/EWIFINQ
指數日行情,TWN/EWIPRCD
指數屬性表,TWN/EWIPRCSTD
證券屬性表,TWN/EWNPRCSTD
證券交易資料表,TWN/EWPRCD
報酬率資訊表,TWN/EWPRCD2
月營收資料表,TWN/EWSALE
三大法人資料,TWN/EWTINST1


In [4]:
pstage = TWMarketMonitor().get_data_from_mongo('PSTAGE', 'tsx', limit = 10000)
pstage 

,_id,指數碼,日期,TEJ本益比,TEJ淨值市價比,成交量(千股),指數收盤價,指數最低價,指數最高價,指數開盤價,流通股數(千股),現金股利率
0,660c9c5060b8c294fc69addd,IR0059,2024-04-02,NaN,NaN,0.0,276.83,NaN,NaN,NaN,0,NaN
1,660c9c5060b8c294fc69adf1,IX0021,2024-04-02,43.3509,3.2446,36479.0,71.69,71.32,71.97,71.42,6699804,1.8723
2,660c9c5060b8c294fc69ae03,IX0039,2024-04-02,17.7019,1.2876,269040.0,1825.18,1819.67,1827.11,1826.10,249490512,2.7070
3,660c9c5060b8c294fc69add3,IR0049,2024-04-02,NaN,NaN,0.0,305.08,NaN,NaN,NaN,0,NaN
4,660c9c5060b8c294fc69adeb,IX0012,2024-04-02,325.6109,1.0823,49557.0,194.03,192.38,194.43,194.43,31820088,1.6445
...,...,...,...,...,...,...,...,...,...,...,...,...
9995,66052db938c6a73d3962606c,IX0001,2010-09-10,17.9135,1.7562,4196331.0,7890.11,7859.33,7911.66,7900.56,581197914,3.4914
9996,66052db938c6a73d3962606b,IX0001,2010-09-09,17.7905,1.7442,4315208.0,7835.54,7817.87,7903.47,7901.83,580973821,3.5159
9997,66052db938c6a73d3962606a,IX0001,2010-09-08,17.8284,1.7479,4233877.0,7851.31,7824.65,7898.76,7866.35,580928031,3.5088
9998,66052db938c6a73d39626069,IX0001,2010-09-07,17.9070,1.7556,4494203.0,7884.40,7876.67,7922.07,7914.38,580773016,3.4942


In [2]:
collection = 'monthly_revenue'
pstage = TWMarketMonitor().get_data_from_mongo('PSTAGE', collection, limit = 500000)
pdata = TWMarketMonitor().pdata_pipline(pstage, collection)
pmart = TWMarketMonitor().pmart_pipline(pdata)
pmart

Unsupported data code. Please use one of the following data codes with the correct column names to rename:
{'融資券資料': 'TWN/EWGIN', '單季財務資料表': 'TWN/EWIFINQ', '指數日行情': 'TWN/EWIPRCD', '指數屬性表': 'TWN/EWIPRCSTD', '證券屬性表': 'TWN/EWNPRCSTD', '證券交易資料表': 'TWN/EWPRCD', '報酬率資訊表': 'TWN/EWPRCD2', '月營收資料表': 'TWN/EWSALE', '三大法人資料': 'TWN/EWTINST1'}


KeyError: "None of ['datetime', 'symbol'] are in the columns"

In [16]:
# pstage = TWMarketMonitor().get_data_from_mongo('PSTAGE', 'securities_returns', limit = 500000)
# pdata = TEJHandler().pdata_pipline(pstage, 'TWN/EWPRCD2')
# pmart = TEJHandler().pmart_pipline(pdata, 'TWN/EWPRCD2')
# pmart

{'日報酬率(%)': symbol      0015    0029    0050    0051    0052    0053    0055    0056  \
 datetime                                                                   
 2022-11-28   NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
 2022-11-29   NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
 2022-11-30   NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
 2022-12-01   NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
 2022-12-02   NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
 ...          ...     ...     ...     ...     ...     ...     ...     ...   
 2024-03-27   0.0  0.0000  0.0317  0.3201 -0.1255  0.4103  0.0000  0.7900   
 2024-03-28   0.0  0.0000 -0.4757  0.0638 -0.2200 -0.4086 -0.4743  0.4804   
 2024-03-29   0.0  0.0000  0.6373  0.1913  0.3465  0.6448  0.7943  0.3020   
 2024-04-01   0.0 -0.9422 -0.3167  0.4456 -0.3139 -0.6407 -0.2364 -0.6774   
 2024-04-02   0.0  1.2581  1.4295  0.0000  2.0151  1.8757 -0.2765

In [19]:
pmart['日報酬率(%)'].sort_index(ascending=False)

symbol,0015,0029,0050,0051,0052,0053,0055,0056,0057,0061,...,IX0059,IX0060,IX0075,IX0085,IX0099,IX0118,MSCI,T1001Y,T1010Y,T1012Y
datetime,,,,,,,,,,,,,,,,,,,,,
2024-04-02,0.0,1.2581,1.4295,0.0000,2.0151,1.8757,-0.2765,0.7325,1.3775,-0.3490,...,-0.1669,0.4918,1.6493,0.6206,1.8698,0.2900,1.6478,0.0,1.2581,0.0
2024-04-01,0.0,-0.9422,-0.3167,0.4456,-0.3139,-0.6407,-0.2364,-0.6774,-0.5991,1.7762,...,0.0191,0.2218,-0.1288,-0.1017,2.5392,0.7999,-0.8808,0.0,-0.9422,0.0
2024-03-29,0.0,0.0000,0.6373,0.1913,0.3465,0.6448,0.7943,0.3020,0.6894,-0.2363,...,0.4311,1.2910,0.9850,1.2753,0.4914,0.8616,1.0194,0.0,0.0000,0.0
2024-03-28,0.0,0.0000,-0.4757,0.0638,-0.2200,-0.4086,-0.4743,0.4804,-0.3435,0.4152,...,-0.9865,-0.3341,2.3346,-0.2398,-1.1167,-0.2892,-0.3761,0.0,0.0000,0.0
2024-03-27,0.0,0.0000,0.0317,0.3201,-0.1255,0.4103,0.0000,0.7900,-0.0429,0.3571,...,0.8659,1.4313,2.5352,0.7345,2.1197,1.1573,0.2073,0.0,0.0000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-12-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-11-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
